## Install tensorflow dependencies

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam

## Install all other dependencies

In [38]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

## Train model

In [39]:
df = pd.read_csv('./../../../timeseries/combined/combined_timeseries_final.csv')
df.fillna(df.mean(), inplace=True)

In [40]:
features_to_be_dropped = ['Timestamp','Durchschnittliche Antwortzeitintervalle','network_outgoing_pod-pod-1', 'network_outgoing_pod-pod-3',
       'network_outgoing_pod-pod-4', 'network_outgoing_pod-pod-5',
       'network_outgoing_pod-pod-9', 'network_outgoing_pod-pod-10',
       'network_outgoing_pod-pod-11', 'network_outgoing_system',
       'cpu_pod-pod-1', 'cpu_pod-pod-2', 'cpu_pod-pod-3', 'cpu_pod-pod-4',
       'cpu_pod-pod-5', 'cpu_pod-pod-6', 'cpu_pod-pod-9', 'cpu_pod-pod-11',
       'cpu_pod-pod-12', 'cpu_pod-pod-13', 'pod-restart-count-pod-1', 'pod-restart-count-pod-2',
       'pod-restart-count-pod-3', 'pod-restart-count-pod-4',
       'pod-restart-count-pod-5', 'pod-restart-count-pod-6',
       'pod-restart-count-pod-8', 'pod-restart-count-pod-10',
       'pod-restart-count-pod-11', 'pod-restart-count-pod-12',
       'pod-restart-count-pod-13', 'network_outgoing_pod-pod-14', 'cpu_pod-pod-14',
       'network_outgoing_pod-pod-15', 'network_outgoing_pod-pod-16',
       'network_outgoing_pod-pod-17', 'network_outgoing_pod-pod-18',
       'network_outgoing_pod-pod-19', 'network_outgoing_pod-pod-20',
       'network_outgoing_pod-pod-21', 'network_outgoing_pod-pod-22',
       'network_outgoing_pod-pod-23', 'network_outgoing_pod-pod-24',
       'network_outgoing_pod-pod-25', 'cpu_pod-pod-15', 'cpu_pod-pod-16',
       'cpu_pod-pod-17', 'cpu_pod-pod-18', 'cpu_pod-pod-19', 'cpu_pod-pod-20',
       'cpu_pod-pod-21', 'cpu_pod-pod-22', 'cpu_pod-pod-23', 'cpu_pod-pod-24',
       'cpu_pod-pod-25', 'pod-restart-count-pod-14',
       'pod-restart-count-pod-15', 'pod-restart-count-pod-16',
       'pod-restart-count-pod-17', 'pod-restart-count-pod-18',
       'pod-restart-count-pod-19', 'pod-restart-count-pod-20',
       'pod-restart-count-pod-21', 'pod-restart-count-pod-22',
       'pod-restart-count-pod-23', 'pod-restart-count-pod-24',
       'pod-restart-count-pod-25']
X = df.drop(features_to_be_dropped, axis=1)
y = df['Durchschnittliche Antwortzeitintervalle']

In [41]:
X.head(5)

,Requests je Sekunde,i_o_read,i_o_write,memory,network_outgoing_pod-pod-2,network_outgoing_pod-pod-6,network_outgoing_pod-pod-7,network_outgoing_pod-pod-8,network_outgoing_pod-pod-12,network_outgoing_pod-pod-13,...,memory_rolling_avg_5,cpu_pod-pod-1_rolling_avg_5,cpu_pod-pod-11_rolling_avg_5,cpu_pod-pod-8_rolling_avg_5,cpu_system_cumsum,memory_cumsum,network_outgoing_system_log_scaled,memory_log_scaled,cpu_system_log_scaled,pod_restart
0,0,0.0,0.0,1.128975e+10,139.0,1001.0,4581.0,143.0,67.3,227.0,...,1.223895e+10,0.019924,0.049688,0.100637,0.00180,1.128975e+10,8.931948,23.147161,0.001798,0.0
1,0,0.0,0.0,1.128867e+10,139.0,1001.0,4548.0,158.0,67.3,229.0,...,1.223895e+10,0.019924,0.049688,0.100637,0.00366,2.257842e+10,8.929700,23.147066,0.001858,0.0
2,0,0.0,0.0,1.128932e+10,139.0,1001.0,4516.0,173.0,67.3,229.0,...,1.223895e+10,0.019924,0.049688,0.100637,0.00529,3.386774e+10,8.932609,23.147123,0.001629,0.0
3,0,0.0,0.0,1.129266e+10,139.0,2466.0,4483.0,188.0,67.3,227.0,...,1.223895e+10,0.019924,0.049688,0.100637,0.00695,4.516039e+10,9.107421,23.147418,0.001659,0.0
4,7,0.0,0.0,1.129897e+10,134.0,2466.0,4451.0,202.0,67.3,227.0,...,1.129187e+10,0.001896,0.001618,0.006194,0.00865,5.645937e+10,9.104980,23.147978,0.001699,0.0


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Daten skalieren

In [43]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Create simple dense modell with tensorflow and keras

In [44]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])


/Users/simonunterlugauer/Desktop/masterarbeit-cluster/performance-prediction/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
model.compile(optimizer=Adam(learning_rate=0.03), loss='mean_squared_error')

In [46]:
model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=300)

Epoch 1/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 5765541.0000 - val_loss: 4077359.0000
Epoch 2/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - loss: 3934643.7500 - val_loss: 2762163.0000
Epoch 3/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - loss: 3489739.5000 - val_loss: 4669411.0000
Epoch 4/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - loss: 3812864.5000 - val_loss: 2981382.7500
Epoch 5/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step - loss: 2373723.2500 - val_loss: 2830379.0000
Epoch 6/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - loss: 3166361.7500 - val_loss: 3111382.7500
Epoch 7/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step - loss: 3319328.7500 - val_loss: 3754198.0000
Epoch 8/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step - loss: 1739083.0000 - val_loss: 2953308.2500
Epoch 9/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step - loss: 2298217.2500 - val_loss: 3641620.2500
Epoch 10/300
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - loss: 2726195.7500 - val_loss: 3489

In [47]:
model.evaluate(X_test_scaled, y_test)

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step - loss: 657618.4375


773622.25

In [48]:
from sklearn.metrics import r2_score

# Machen Sie Vorhersagen mit Ihrem Modell
y_pred = model.predict(X_test_scaled)

# Berechnen Sie das R²-Maß
r2 = r2_score(y_test, y_pred)

print("R²-Maß:", r2)

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
R²-Maß: 0.8933157703824531
